In [1]:
#importing necessary libraries
import selenium 
import pandas as pd
from selenium import webdriver
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

# Amazon search

In [46]:
driver=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising chrome

In [47]:
driver.get('https://www.amazon.in/') #going to amazon website

In [48]:
enter_prod=input("Enter the product you are looking for: ") #taking input for searchbar from user

Enter the product you are looking for: airpods


In [49]:
search_amazon=driver.find_element_by_id('twotabsearchtextbox') #selecting seachbar
search_amazon.send_keys(enter_prod) #entering user input in searchbar

In [50]:
hit_search=driver.find_element_by_id('nav-search-submit-button') #selecting search button
hit_search.click() #hitting search

In [51]:
#creating lists to store scraped data
name_of_product=[]
brand_name=[]
price_of_product=[]
return_exchange=[]
availability=[]
expected_delivery=[]
product_url=[]

In [52]:
page=1 
while True: #creating loop to scrape data from first 3 pages
    name=driver.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']") #Scraping name of product
    for val in name:
        name_of_product.append(val.text)
    aim=driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']") #scraping product links
    for val in aim:
        product_url.append(val.get_attribute("href")) #storing only urls of product
    if(page<3): #checking condition of page number before clicking next page
        next_page=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        next_page.click()  #clicking next page
        time.sleep(4)
        page+=1
    else:
        break #breaking loop if page condition does'nt fulfill

In [53]:
#seperatly going to every product link to extract additional info
for val in product_url: #product_url contains all the urls
    try:                                        #try getting price
        driver.get(val)
        time.sleep(3)
        price=driver.find_element_by_xpath("//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']/span[2]")
        price_of_product.append(price.text[0:-2].replace("\n"," ")) #removing unnecessary data
    except: 
        pass  #if one class is not present try another one
        try:
            price=driver.find_element_by_xpath("//span[@class='a-price a-text-price a-size-medium']")
            price_of_product.append(price.text)
        except NoSuchElementException:  #append - in place of price if none of the class is present
            price_of_product.append("-")

In [54]:
for val in product_url:
    try:    #try getting brand names
        driver.get(val)
        time.sleep(4)
        brand=driver.find_element_by_xpath("//tr[@class='a-spacing-small po-brand']/td[2]")
        brand_name.append(brand.text)
    except:    #append - if we dont find it
        brand_name.append("-")
    try:   #try getting availability of the product 
        avail=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        availability.append(avail.text)
    except:
        availability.append("-") #append - if we dont find it
    try:    #try getting return details 
        ret=driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']")
        return_exchange.append(ret.text)
    except:  #append - if we dont find it
        return_exchange.append("-")   
    try:   #try getting delivery details
        delivery=driver.find_element_by_xpath("//div[@id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']/span/span")
        expected_delivery.append(delivery.text)
    except:  #append - if we dont find it
        expected_delivery.append("-")

In [55]:
#checking the length of each list
print(len(expected_delivery),len(product_url),len(name_of_product),len(brand_name),len(price_of_product),len(availability),len(return_exchange))

54 54 54 54 54 54 54


In [56]:
df=pd.DataFrame({"Brand":brand_name,"Product_name":name_of_product,"Price":price_of_product,"Return/Exchange":return_exchange,"Expected_Delivery":expected_delivery,"Availability":availability,"Product_url":product_url})
df #creating dataFrame with our scraped data

,Brand,Product_name,Price,Return/Exchange,Expected_Delivery,Availability,Product_url
0,Apple,Apple AirPods (2nd Generation),"₹10,999",-,"Saturday, 9 July",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Apple,New Apple AirPods Pro with MagSafe Charging Case,"₹18,999",-,"Monday, 11 July",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Apple,New Apple AirPods Pro with MagSafe Charging Case,"₹18,999",-,"Monday, 11 July",In stock.,https://www.amazon.in/Apple-AirPods-MagSafe-Ch...
3,BoAt,"boAt Airdopes 141 42H Playtime, Beast Mode ENx...","₹1,499",7 Days Replacement,"Saturday, 9 July",In stock.,https://www.amazon.in/Airdopes-141-Playtime-Re...
4,Apple,Apple AirPods (2nd Generation),"₹10,999",-,"Saturday, 9 July",In stock.,https://www.amazon.in/Apple-AirPods-with-Charg...
5,Apple,New Apple AirPods (3rd Generation),"₹17,900",-,"Saturday, 9 July",In stock.,https://www.amazon.in/Apple-MME73HN-A-AirPods-...
6,AC Accessories,Air.pods Pro with Wireless Charging Case - White.,₹599,7 Days Replacement,"Tuesday, 12 July",In stock.,https://www.amazon.in/Air-pods-Pro-Wireless-Ch...
7,JMD,Airpod for OnePlus 7T Pro with Wireless Chargi...,"₹1,399",7 Days Replacement Only,"Tuesday, 12 July",In stock.,https://www.amazon.in/OnePlus-7T-Pro-Cancellat...
8,BoAt,Boat Airdopes 121V2 Bluetooth Truly Wireless i...,"₹1,299",7 Days Replacement,"Saturday, 9 July",In stock.,https://www.amazon.in/Airdopes-121v2-Bluetooth...
9,Wastechpro,WasTechPro™ Premium Airpods with Wireless Char...,"₹1,599",7 Days Replacement,"Monday, 11 July",Only 2 left in stock.,https://www.amazon.in/WasTechProTM-Premium-Air...


In [57]:
driver.close() #closing our browser

# Google images

In [5]:
driver2=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising our browser 

In [6]:
driver2.get('https://images.google.com/') #going to google images

In [7]:
#creating list to store scraped data
fruits=[]
cars=[]
ml=[]
guitars=[]
cakes=[]

In [8]:
#creating a big list to store all other empty lists
big_list=[fruits,cars,ml,guitars,cakes]
driver2.maximize_window() #maximizing viewspace
search=driver2.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input") #selecting search bar
search.send_keys('fruits') #entering fruits in the search bar

In [9]:
hit_search=driver2.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button')
hit_search.click() #clicking search button

In [10]:
def scrap_src(i): #creating a function to scrape source of images. i refers to index of big list 
    image=driver2.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    for val in image:
        big_list[i].append(val.get_attribute('src'))
        big_list[i]=big_list[i][0:10]

In [11]:
scrap_src(0) #selecting fruits from biglist and scraping sources of images

In [12]:
def clear(): #function to clear search bar
    clear_bar=driver2.find_element_by_xpath("//input[@class='og3lId']")
    clear_bar.clear()

In [13]:
def search_input(x): #function to enter keys in search bar
    search_img=driver2.find_element_by_xpath("//input[@class='og3lId']")
    search_img.send_keys(x)

In [14]:
def hit_search_fun(): #function to hit search button
    hit=driver2.find_element_by_xpath("//button[@class='rCGXm']")
    hit.click()

In [15]:
clear() #clearing input bar

In [16]:
search_input('cars') #searching for cars
hit_search_fun() #hitting search button
time.sleep(3) #waiting for page to load
scrap_src(1) #scraping and storing data in cars list which is present in big_list[1]
clear() #clearing search bar

In [17]:
search_input('Machine Learning') #searching for machine learning
hit_search_fun() #hitting search button
time.sleep(3) #waiting for page to load
scrap_src(2) #scraping and storing data in ml list which is present in big_list[2]
clear() #clearing search bar

In [18]:
search_input('Guitar') #searching for guitar
hit_search_fun() #hitting search button
time.sleep(3) #waiting for page to load
scrap_src(3)  #scraping and storing data in guitars list which is present in big_list[3]
clear() #clearing seachbar

In [19]:
search_input('cakes')  #searching for cakes
hit_search_fun()   #hitting search button
time.sleep(3)  #waiting for page to load
scrap_src(4)  #scraping and storing data in cakes list which is present in big_list[4]
clear()  #clearing seachbar

In [20]:
print(len(big_list[1]),len(big_list[0]),len(big_list[2]),len(big_list[3]),len(big_list[4])) #checking length of each list

10 10 10 10 10


In [21]:
df2=pd.DataFrame({"Fruits":big_list[0],"Cars":big_list[1],"Machine Learning":big_list[2],"Guitar":big_list[3],"Cakes":big_list[4]})
df2 #creating dataframe with our scraped data

,Fruits,Cars,Machine Learning,Guitar,Cakes
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [22]:
driver2.close() #closing our driver

# Flipkart

In [23]:
driver3=webdriver.Chrome('C:\webdriver\chromedriver.exe')  # opening browser

In [24]:
driver3.get('https://www.flipkart.com/') #searching flipkat in our browser

In [25]:
cancel_login=driver3.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']") #selecting not to login
cancel_login.click() 

In [26]:
insert_input=driver3.find_element_by_xpath("//input[@class='_3704LK']")
insert_input.send_keys('one plus 9') #entering oneplus 9 in search bar

In [27]:
search_but=driver3.find_element_by_class_name('L0Z3Pu')
search_but.click() #hitting search button

In [28]:
#creating list to store data
brand_name=[]
smartphone_name=[]
color=[]
ram=[]
rom=[]
pri_camera=[]
sec_camera=[]
display=[]
battery=[]
price=[]
product_url=[]

In [29]:
brand=driver3.find_elements_by_class_name('_4rR01T') #scraping brand name with the help of class name
for val in brand:
    val=val.text
    brand_name.append(val.split()[0])

In [30]:
phone_name=driver3.find_elements_by_class_name('_4rR01T') #scraping phone name with the help of class name
for val in phone_name:
    val=val.text
    smartphone_name.append(val.split("(")[0]) #removing unnecessary strings 
    val=val.split("(",1)[1] # splitting and appending corresponding data to its list
    color.append(val.split(",")[0])
    val=val.split(",")[1]
    rom.append(val.strip(")"))

In [31]:
cost=driver3.find_elements_by_xpath("//div[@class='_30jeq3 _1_WHN1']") #scraping price
for val in cost:
    price.append(val.text)

In [32]:
dis=driver3.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[2]") #scraping display details
for val in dis:
    display.append(val.text)

In [33]:
primary=driver3.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[3]") #scraping details of primary camera
for val in primary: 
    pri_camera.append(val.text)

In [34]:
ram_mem=driver3.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[1]") #scraping ram
for val in ram_mem:
    ram.append(val.text.split("|")[0])

In [35]:
batt=driver3.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[4]") #scraping battery value
for val in batt:
    battery.append(val.text.rsplit(" ",1)[0])

In [36]:
sec_camera=pri_camera #giving same values to secondary camera cause its not available properly

In [37]:
url=driver3.find_elements_by_class_name('_1fQZEK') #scraping product url
for val in url:
    product_url.append(val.get_attribute('href'))

In [38]:
df3=pd.DataFrame({"Brand":brand_name,"Product":smartphone_name,"Color":color,"RAM":ram,"Storage(ROM)":rom,"Primary_camera":pri_camera,"Secondary_camera":sec_camera,"Display_size":display,"Battery_capacity":battery,"Price":price,"Product_url":product_url})

In [41]:
df3["Secondary_camera"]="16MP"  #all the models have the same 16MP of secondary camera but one has 32MP
df3["Secondary_camera"][22:24]="32MP" # The one which has 32MP
df3 

,Brand,Product,Color,RAM,Storage(ROM),Primary_camera,Secondary_camera,Display_size,Battery_capacity,Price,Product_url
0,OnePlus,OnePlus 9RT 5G,Hacker Black,8 GB RAM,128 GB,50MP Rear Camera,16MP,16.81 cm (6.62 inch) Display,4500 mAh,"₹42,450",https://www.flipkart.com/oneplus-9rt-5g-hacker...
1,OnePlus,OnePlus 9 Pro 5G,Pine Green,12 GB RAM,256 GB,48MP Rear Camera,16MP,17.02 cm (6.7 inch) Display,4500 mAh,"₹56,999",https://www.flipkart.com/oneplus-9-pro-5g-pine...
2,OnePlus,OnePlus 9RT 5G,Hacker Black,12 GB RAM,256 GB,50MP Rear Camera,16MP,16.81 cm (6.62 inch) Display,4500 mAh,"₹45,490",https://www.flipkart.com/oneplus-9rt-5g-hacker...
3,OnePlus,OnePlus 9RT 5G,Nano Silver,8 GB RAM,128 GB,50MP Rear Camera,16MP,16.81 cm (6.62 inch) Display,4500 mAh,"₹40,499",https://www.flipkart.com/oneplus-9rt-5g-nano-s...
4,OnePlus,OnePlus 9 5G,Winter Mist,12 GB RAM,256 GB,48MP Rear Camera,16MP,16.64 cm (6.55 inch) Display,4500 mAh,"₹42,899",https://www.flipkart.com/oneplus-9-5g-winter-m...
5,OnePlus,OnePlus 9 Pro 5G,Morning Mist,8 GB RAM,128 GB,48MP Rear Camera,16MP,17.02 cm (6.7 inch) Display,4500 mAh,"₹48,949",https://www.flipkart.com/oneplus-9-pro-5g-morn...
6,OnePlus,OnePlus 9 Pro,Pine Green,8 GB RAM,128 GB,48MP Rear Camera,16MP,17.02 cm (6.7 inch) Display,4500 mAh,"₹49,299",https://www.flipkart.com/oneplus-9-pro-pine-gr...
7,OnePlus,OnePlus 9 5G,Astral Black,12 GB RAM,256 GB,48MP Rear Camera,16MP,16.81 cm (6.62 inch) Display,4500 mAh,"₹42,799",https://www.flipkart.com/oneplus-9-5g-astral-b...
8,OnePlus,OnePlus 9RT 5G,Nano Silver,12 GB RAM,256 GB,50MP Rear Camera,16MP,16.81 cm (6.62 inch) Display,4500 mAh,"₹44,999",https://www.flipkart.com/oneplus-9rt-5g-nano-s...
9,OnePlus,OnePlus 9 5G,Arctic Sky,12 GB RAM,256 GB,48MP Rear Camera,16MP,16.64 cm (6.55 inch) Display,4500 mAh,"₹42,500",https://www.flipkart.com/oneplus-9-5g-arctic-s...


In [42]:
driver3.close() #closing our browser

# Google Maps

In [272]:
driver4=webdriver.Chrome('C:\webdriver\chromedriver.exe') # initialsing chrome browser

In [273]:
driver4.get('https://www.google.co.in/maps') # Opening Google Maps on our chrome 

In [274]:
city="New York" #giving newyork as city
enter_city=driver4.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]")
enter_city.send_keys(city) #entering city in search bar

In [275]:
hit_search=driver4.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button')
hit_search.click() #clicking search
time.sleep(3)

In [276]:
geo=driver4.current_url #The coordinates are always available in url so storing url and removing unnecessary details
geog=geo.split("@")[1]
geog=geog.split(",")[0:2]

In [279]:
latitude=geog[0] #storing corresponding values 
longitude=geog[1]
print(latitude,longitude)

40.6976701 -74.2598801


In [281]:
df4=pd.DataFrame({"City":city,"Latitude":latitude,"Longitude":longitude},index=[0])
df4 

,City,Latitude,Longitude
0,New York,40.6976701,-74.2598801


In [282]:
driver4.close() #closing browser

# Funding Deals for second quarter (i.e Jan 21 – March 21)

In [3]:
driver5=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising our web browser

In [4]:
driver5.get('https://trak.in/') #going to trak.in website

In [5]:
driver5.maximize_window() #fullscreen window 

In [6]:
deals=driver5.find_element_by_xpath('/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a')
deals.click() #clicking on funding deals

In [ ]:
#Now you have to manually close the ad window

In [50]:
dates=[] #creating list to store all the dates 
date=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[2]")
for val in date:
    dates.append(val.text)
dates=dates[-364:-388:-1] #storing dates of only Jan21 - March21

In [27]:
start_up=[] #creating list to store all the startup names
start=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[3]")
for val in start:
    start_up.append(val.text)
start_up=start_up[-364:-388:-1] #storing data of only Jan21 - March21

In [31]:
industry=[] #creating list to store all the industry names
indus=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[4]")
for val in indus:
    industry.append(val.text)
industry=industry[-364:-388:-1] #storing data of only Jan21 - March21

In [38]:
sub_vertical=[] #creating list to store all the sub_verticals
sub=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[5]")
for val in sub:
    sub_vertical.append(val.text)
sub_vertical=sub_vertical[-364:-388:-1] #storing data of only Jan21 - March21

In [43]:
city_loc=[] #creating list to store all the locations
city=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[6]")
for val in city:
    city_loc.append(val.text)
city_loc=city_loc[-364:-388:-1] #storing data of only Jan21 - March21

In [44]:
investors=[] #creating list to store all the investors names
invest=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[7]")
for val in invest:
    investors.append(val.text)
investors=investors[-364:-388:-1] #storing data of only Jan21 - March21

In [45]:
investment_type=[] #creating list to store all the investment type
invest_type=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[8]")
for val in invest_type:
    investment_type.append(val.text)
investment_type=investment_type[-364:-388:-1] #storing data of only Jan21 - March21

In [ ]:
amount=[] #creating list to store the amount of funding 
amt=driver5.find_elements_by_xpath("//tbody[@class='row-hover']/tr/td[9]")
for val in amt:
    amount.append(val.text)
amount=amount[-364:-388:-1] #storing data of only Jan21 - March21

In [73]:
df5=pd.DataFrame({"Date":dates,"Start_up_name":start_up,"Industry/Vertical":industry,"Sub_vertical":sub_vertical,"City/Location":city_loc,"Investors'_Name":investors,"Investment_type":investment_type,"Amount":amount})
df5

,Date,Start_up_name,Industry/Vertical,Sub_vertical,City/Location,Investors'_Name,Investment_type,Amount
0,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
1,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
2,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
3,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
5,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
6,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
7,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
8,09/02/2021,SplashLearn,EdTech,Game-based learning programme,Gurgaon,Owl Ventures,Series C,"18,000,000"
9,09/02/2021,Nothing,Technology,Consumer Technology Venture,London,GV,Series A,"15,000,000"


In [74]:
driver5.close() #closing browser

# Gaming Laptops 

In [97]:
driver6=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising chrome browser

In [98]:
driver6.get('https://www.digit.in/') #searching digit.in

In [78]:
driver6.maximize_window() #full window

In [99]:
open_search=driver6.find_element_by_xpath("//div[@class='search']")
open_search.click() #clicking on search bar

In [102]:
enter_search=driver6.find_element_by_xpath("/html/body/div[2]/div[2]/div/div/input")
enter_search.send_keys("Best gaming laptops") #entering data on search bar

In [103]:
enter_search.send_keys(Keys.ENTER) #hitting enter 

In [104]:
#creating lists to store data
laptop_names=[]
laptop_price=[]
digit_rating=[]

In [105]:
laptop=driver6.find_elements_by_xpath("//div[@class='searchProduct-desc']") #scraping laptop names
for val in laptop:
    laptop_names.append(val.text)

In [106]:
lap_urls=[] #scraping urls of each product
urls=driver6.find_elements_by_xpath("//div[@class='searchPage']/a") 
for val in urls:
    lap_urls.append(val.get_attribute('href'))

In [107]:
for val in lap_urls: #scraping data for each product
    driver6.get(val) #going to product's window to scrape data
    time.sleep(3) #waiting for page to load
    try: #try getting laptop price 
        price=driver6.find_element_by_xpath("//div[@class='price']/h2/strong")
        laptop_price.append(price.text)
    except: #if price is not available append - to the list
        laptop_price.append("-")
    try: #try getting laptop rating
        rat=driver6.find_element_by_xpath("//span[@class='rating_no greentext' or @class='rating_no yellowtext']")
        digit_rating.append(rat.text)
    except: #if rating is not available append - to the list
        digit_rating.append("-")

In [ ]:
brand_names=[] #scraping brand names and removing unnecessary data
for val in laptop_names:
    brand_names.append(val.split()[0])

In [120]:
df6=pd.DataFrame({"Brand":brand_names,"Name of product":laptop_names,"Price":laptop_price,"Digit_rating":digit_rating})
df6 #creating dataframe with our scraped data

,Brand,Name of product,Price,Digit_rating
0,Asus,Asus G750JX-CV069P,"₹ 149,999",-
1,Lenovo,Lenovo G50-30,"₹ 25,694",-
2,ASUS,ASUS G53J - A 3D gamer's delight but we want more,"₹ 112,499",70
3,HP,HP Envy dv6-7206tx notebook,"₹ 65,705",90
4,Dell,Dell Vostro 15 3500,"₹ 38,489",-
5,HP,HP Envy 15-k006tx,"₹ 80,510",-
6,Sony,Sony PlayStation Vita,"₹ 24,990",80
7,ASUS,ASUS ROG Flow x13,"₹ 240,490",81
8,Lenovo,Lenovo IdeaPad Y570,"₹ 49,900",90
9,Samsung,Samsung Series 9 (NP-900X3C),-,90


In [ ]:
driver6.close() #closing browser

# Forbes Billionaires list


In [2]:
driver7=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising browser

In [3]:
driver7.get('https://www.forbes.com/billionaires/') #going to forbes website

In [4]:
time.sleep(10) #waiting for it to load

In [28]:
#creating lists to store data
rank=[]
names=[]
age=[]
net_worth=[]
source=[]
nationality=[]
industry=[]

In [ ]:
page=1 #creating loop to collect data from all the pages 
while True:
    ranks=driver7.find_elements_by_xpath("//div[@class='rank']") #scraping ranks
    for val in ranks:
        rank.append(val.text)
    name=driver7.find_elements_by_xpath("//div[@class='personName']") #scraping names
    for val in name:
        names.append(val.text)
    worth=driver7.find_elements_by_xpath("//div[@class='netWorth']") #scraoing net worth
    for val in worth:
        net_worth.append(val.text)
    ag=driver7.find_elements_by_xpath("//div[@class='age']") #scraping age
    for val in ag: 
        age.append(val.text)
    country=driver7.find_elements_by_xpath("//div[@class='countryOfCitizenship']") #scraping nationality
    for val in country:
        nationality.append(val.text)
    src=driver7.find_elements_by_xpath("//div[@class='expand-row__icon-container']") #scraping source of income
    for val in src:
        source.append(val.text)   
    ind=driver7.find_elements_by_xpath("//div[@class='category']/div") #scraping their industries
    for val in ind:
        industry.append(val.text)
    p+=1 #increasing page number 
    if(page<14):
        next_button=driver7.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click() #going to next page 
        time.sleep(10) #waiting for data to load
    else:
        break

In [30]:
print(len(industry),len(names),len(age),len(source),len(nationality),len(net_worth),len(rank)) #checking length of each list

2668 2668 2668 2668 2668 2668 2668


In [32]:
df7=pd.DataFrame({"Rank":rank,"Name":names,"Net_worth":net_worth,"Age":age,"Nationality":nationality,"Source":source,"Industry":industry})
df7 #creating dataframe with our scraped data

,Rank,Name,Net_worth,Age,Nationality,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
5,6.,Larry Page,$111 B,49,United States,Google,Technology
6,7.,Sergey Brin,$107 B,48,United States,Google,Technology
7,8.,Larry Ellison,$106 B,77,United States,software,Technology
8,9.,Steve Ballmer,$91.4 B,66,United States,Microsoft,Technology
9,10.,Mukesh Ambani,$90.7 B,64,India,diversified,Diversified


In [33]:
driver7.close() #closing driver

# Youtube Comments

In [234]:
driver8=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising chrome browser

In [235]:
driver8.get('https://www.youtube.com/watch?v=9bZkp7q19f0') #go to youtube

In [249]:
#creating list to store data
comments=[]
upvotes=[]
posted=[]

In [251]:
page_scroll=driver8.find_element_by_tag_name("body") #selecting the body of the webpage

In [258]:
while len(comments)<501: #comments are less than 501 
    page_scroll.send_keys(Keys.PAGE_DOWN) #scrolling down to load more data
    time.sleep(3) #waiting for comments to load
    comment=driver8.find_elements_by_xpath("//ytd-expander[@class='style-scope ytd-comment-renderer']")
    for val in comment:
        comments.append(val.text.replace("\n"," ")) #storing comments 
    likes=driver8.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer'][2]")
    for val in likes:
        upvotes.append(val.text) #storing amounts of likes the comment has got
    post=driver8.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text style-scope ytd-comment-renderer']")
    for val in post:
        posted.append(val.text) #scraping the time when the comment was posted

In [259]:
len(comments),len(upvotes),len(posted)

(540, 540, 540)

In [260]:
df8=pd.DataFrame({"Comments":comments,"Likes":upvotes,"Posted":posted})
df8

,Comments,Likes,Posted
0,"Lets be honest here, this guy owned the world ...",126K,2 years ago (edited)
1,This song never gets old. No matter how much I...,4K,12 days ago
2,"10 years later, I still have no idea what this...",5.7K,2 weeks ago
3,Didn't realize we have been listening to kpop ...,222K,2 years ago (edited)
4,"This guy appeared out of nowhere, and everyone...",35K,2 years ago
5,"Ааааааа, я это в 3 года слушала , когда мама е...",7,6 hours ago (edited)
6,I dont think people realize just how much this...,3.2K,2 months ago
7,"It’s 2022, let’s see how many legends are list...",1.4K,12 days ago
8,Saudades dessa época! Aeee cadê os BR??? 🇧🇷🇧🇷🇧...,24K,2 years ago
9,Can't believe this song is almost 10 years old...,3.3K,2 weeks ago


In [261]:
driver8.close() #closing browser

# Hostel World

In [173]:
driver9=webdriver.Chrome('C:\webdriver\chromedriver.exe') #initialising web browser

In [174]:
driver9.get('https://www.hostelworld.com/') #going to hostelworld website
time.sleep(15) #waiting for page to load properly

In [178]:
search_input=driver9.find_element_by_xpath("//input[@class='search-input']") 
search_input.send_keys("london") #entering london in search bar
time.sleep(3)

In [180]:
click_london=driver9.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div")
click_london.click() #clicking search

In [181]:
click_search_button=driver9.find_element_by_xpath("//button[@class='button primary large']")
click_search_button.click() #hitting search
time.sleep(5)

In [184]:
check_in=driver9.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[2]/div/div/div/div[2]/div") #selecting check-in date
check_in.click() 

In [185]:
next_month=driver9.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[2]/div/div/div/div[2]/div[2]/i[2]")
next_month.click() #selecting month

In [186]:
select_date=driver9.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[2]/div/div/div/div[2]/div[2]/div/div[13]/div")
select_date.click() #selecting date

In [187]:
check_out=driver9.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[2]/div/div/div/div[3]/div[2]/div/div[14]/div")
check_out.click() #selecting checkout date

In [188]:
hit_search=driver9.find_element_by_xpath("//button[@class='button primary large full-width']")
hit_search.click() #searching for hostels 

In [189]:
#creating list to store data
hostel_name=[]
distance=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
privates=[]
dorms=[]
facilities=[]
description=[]
all_facility=[]

In [190]:
hostel=driver9.find_elements_by_xpath("//h2[@class='title title-6']") #scraping hostels name
for val in hostel:
    hostel_name.append(val.text) 

In [191]:
total_rev=driver9.find_elements_by_xpath("//div[@class='reviews']") #scraping reviews
for val in total_rev:
    total_reviews.append(val.text)

In [ ]:
rooms=[]
pri=driver9.find_elements_by_xpath("//div[@class='price-col']") #scraping prices for dorms and private rooms
for val in pri:
    val=val.text.replace("\n"," ")
    if val.count("R") >0: #to scrape proper price
        rooms.append(val.split(" ")[-1])
    else:
        rooms.append(val)

In [195]:
links=[]
link=driver9.find_elements_by_xpath("//h2[@class='title title-6']/a") #storing urls of each results 
for val in link:
    links.append(val.get_attribute('href'))

In [196]:
dist=driver9.find_elements_by_xpath("//span[@class='description']") #storing distance from city center
for val in dist:
    distance.append(val.text.split("-")[1])

In [223]:
for val in links:
    driver9.get(val) #going to every results and scraping data from it
    time.sleep(4)
    descript=driver9.find_element_by_xpath("//div[@class='content collapse-content']") #scraping description
    description.append(descript.text)
    rating=driver9.find_element_by_xpath("//div[@class='rating rating-summary-container big clickable']") #scraping reviews
    overall_reviews.append(rating.text.split('\n')[0])
    fac=driver9.find_elements_by_xpath("//ul[@class='facilities']/li") #scraping facilities
    for val in fac:
        facilities.append(val.text)
    all_facility.append(facilities)
    try:  #try scrapping all ratings if anything is missing append("-")
        sep_rat=driver9.find_element_by_xpath("//ul[@class='rating-breakdown rating-grid']")
        ratings.append(sep_rat.text.replace("\n"," "))
    except:
        ratings.append("-")
        

In [224]:
privates=rooms[0::2] #separating prices of privates and dorms
dorms=rooms[1::2]

In [231]:
print(len(dorms),len(privates))

17 17


In [232]:
df9=pd.DataFrame({"Hostel/Hotel Name":hostel_name,"Location":distance,"Ratings":ratings,"Total_reviews":total_reviews,"Overall_reviews":overall_reviews,"Private_price":privates,"Dorms_price":dorms,"Facilities":all_facility,"Property_Description":description})
df9 #creating dataframe from our scraped data 

,Hostel/Hotel Name,Location,Ratings,Total_reviews,Overall_reviews,Private_price,Dorms_price,Facilities,Property_Description
0,St Christopher's Village,1.8km from city centre,Security 8.3 Location 9 Staff 8.2 Atmosphere 7...,11307 Total Reviews,8.0,No Privates Available,Rs7307,"[Linen Included, Free City Maps, Free WiFi, Fr...",COVID 19 Policy Update.\nIn response to Corona...
1,Safestay London Elephant & Castle,1.7km from city centre,Security 7.7 Location 7.7 Staff 6.8 Atmosphere...,4349 Total Reviews,7.0,No Privates Available,Rs10147,"[Linen Included, Free City Maps, Free WiFi, Fr...",Safestay at Elephant & Castle is ideal if you ...
2,Safestay London Kensington Holland Park,5.9km from city centre,Security 7.1 Location 7.8 Staff 6.7 Atmosphere...,1236 Total Reviews,6.4,No Privates Available,Rs9974,"[Linen Included, Free City Maps, Free WiFi, Fr...",Safestay Holland Park\n\nSafestay Holland Park...
3,Hostel One Notting Hill,5.5km from city centre,Security 9.3 Location 9.3 Staff 9.6 Atmosphere...,1386 Total Reviews,9.1,Rs34246,No Dorms Available,"[Linen Included, Free City Maps, Free WiFi, Fr...",The perfect place for solo travelers to connec...
4,St Christopher's Hammersmith,7.5km from city centre,Security 8.6 Location 9 Staff 8.2 Atmosphere 7...,4046 Total Reviews,8.0,No Privates Available,Rs8270,"[Linen Included, Free City Maps, Free WiFi, Fr...","In response to Coronavirus (COVID-19), additio..."
5,"PubLove @ The Steam Engine, Waterloo",0.5km from city centre,Security 9.2 Location 9.3 Staff 9 Atmosphere 7...,277 Total Reviews,8.4,No Privates Available,Rs7892,"[Linen Included, Free City Maps, Free WiFi, Fr...",The heart & soul of London backpacking\nPull u...
6,YHA London Earls Court,5.1km from city centre,Security 9.9 Location 9.7 Staff 9.7 Atmosphere...,1412 Total Reviews,9.5,Rs19178,No Dorms Available,"[Linen Included, Free City Maps, Free WiFi, Fr...",Temporary changes at YHA London Earl's Court\n...
7,Saint James Backpackers,5.5km from city centre,Security 7.2 Location 8.1 Staff 8.1 Atmosphere...,1807 Total Reviews,6.9,Rs27013,No Dorms Available,"[Linen Included, Free City Maps, Free WiFi, Fr...",We are located in central London - just 5 minu...
8,"PubLove @ The Green Man, Paddington",4.3km from city centre,Security 8.5 Location 8.8 Staff 8.9 Atmosphere...,414 Total Reviews,8.0,No Privates Available,Rs6851,"[Linen Included, Free City Maps, Free WiFi, Fr...",Where it all began - PubLove was born here in ...
9,Britannia Inn Hotel,14.5km from city centre,Security 9 Location 7 Staff 9 Atmosphere 8 Cle...,21 Total Reviews,8.1,Rs48086,No Dorms Available,"[Linen Included, Free City Maps, Free WiFi, Fr...",The Britannia Inn Hotel is situated near the I...


In [233]:
driver9.close() #closing our browser